Gcode45

In [1]:
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 

In [2]:
#Parameter
model = AbstractModel()
model.gen=Set()
model.t=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.d=Param(model.gen)
model.e=Param(model.gen)
model.f=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)
model.RU=Param(model.gen)
model.RD=Param(model.gen)

model.teta=Param(model.t)
model.demand=Param(model.t)
model.wind=Param(model.t)


#Scalar
model.Elim=Param(initialize=float('inf'),mutable=True)

In [3]:
# Limitation of P(g,t)
def Pbounds(model,gen,t):
  return (model.Pmin[gen],model.Pmax[gen])

# Variable
model.p=Var(model.gen,model.t,bounds=Pbounds,within=Reals)
model.cost=Var(within=Reals)
model.EM=Var(within=Reals)
model.of=Var(within=Reals)

#Constraint
def rule2(model,gen,t):
    if t==1:
      return model.p[gen,t+1] - model.p[gen,t] <= model.RU
    else:
        return Constraint.Skip
model.c2 = Constraint(model.gen,model.t,rule=rule2)

def rule3(model,gen,t):
    if t==1:
      return model.p[gen,t-1]- model.p[gen,t] <= model.RD
    else:
        return Constraint.Skip
model.c3 = Constraint(model.gen,model.t,rule=rule3)

def rule6(model):
  return model.EM == sum((model.d[gen]*model.p[gen,t]*model.p[gen,t] + 
                              model.e[gen]*model.p[gen,t]+model.f[gen]) for gen in model.gen for t in model.t) 
model.c5 = Constraint(rule=rule6)

def ruleTC(model):
  return model.cost == sum((model.a[gen]*model.p[gen,t]*model.p[gen,t] + 
                              model.b[gen]*model.p[gen,t]+model.c[gen]) for gen in model.gen for t in model.t) 
model.c4 = Constraint(rule=ruleTC)

def rule8(model,t):
    return sum(model.p[gen,t] for gen in model.gen ) <= model.demand[t]
model.c8 = Constraint(model.t,rule=rule8)

def rule9(model):
  return model.EM <= model.Elim
model.c9 = Constraint(rule=rule9)

#objective function
def of(model):
    return model.of == sum((1*model.teta[t]*model.p[gen,t]) for gen in model.gen for t in model.t) - model.cost
model.c7 = Constraint(rule=of)



In [4]:
# Calling function min or max
model.obj=Objective(expr=model.of,sense=maximize)

In [5]:
instance= model.create_instance("code44.dat")

In [6]:
results = SolverFactory('ipopt').solve(instance).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Lower bound: -inf
  Upper bound: inf
  Number of objectives: 1
  Number of constraints: 27
  Number of variables: 99
  Sense: unknown
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Message: Ipopt 3.14.10\x3a Optimal Solution Found
  Termination condition: optimal
  Id: 0
  Error rc: 0
  Time: 0.17323088645935059
# ----------------------------------------------------------
#   Solution Information
# ----------------------------------------------------------
Solution: 
- number of solutions: 0
  number of solutions displayed: 0


In [7]:
instance.of.pprint()

of : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :  None : 99587.64100154815 :  None : False : False :  Reals


In [8]:
instance.cost.pprint()

cost : Size=1, Index=None
    Key  : Lower : Value              : Upper : Fixed : Stale : Domain
    None :  None : 241078.00295221675 :  None : False : False :  Reals


In [9]:
instance.EM.pprint()

EM : Size=1, Index=None
    Key  : Lower : Value             : Upper : Fixed : Stale : Domain
    None :  None : 965579.3800655528 :  None : False : False :  Reals


In [10]:
instance.p.pprint()

p : Size=96, Index=gen*t
    Key           : Lower : Value              : Upper : Fixed : Stale : Domain
     ('G1', 't1') :    28 :  74.62500000010442 :   200 : False : False :  Reals
    ('G1', 't10') :    28 : 135.24999999990365 :   200 : False : False :  Reals
    ('G1', 't11') :    28 : 121.95833333328305 :   200 : False : False :  Reals
    ('G1', 't12') :    28 :  99.29166666668685 :   200 : False : False :  Reals
    ('G1', 't13') :    28 : 104.62500000000301 :   200 : False : False :  Reals
    ('G1', 't14') :    28 : 102.70833333333906 :   200 : False : False :  Reals
    ('G1', 't15') :    28 : 109.74999999998258 :   200 : False : False :  Reals
    ('G1', 't16') :    28 : 101.79166666667658 :   200 : False : False :  Reals
    ('G1', 't17') :    28 : 155.49999999980338 :   200 : False : False :  Reals
    ('G1', 't18') :    28 : 108.54166666665807 :   200 : False : False :  Reals
    ('G1', 't19') :    28 : 109.99999999998488 :   200 : False : False :  Reals
     ('G1', 't2

In [12]:
instance.pprint()

2 Set Declarations
    gen : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {'G1', 'G2', 'G3', 'G4'}
    t : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   24 : {'t1', 't2', 't3', 't4', 't5', 't6', 't7', 't8', 't9', 't10', 't11', 't12', 't13', 't14', 't15', 't16', 't17', 't18', 't19', 't20', 't21', 't22', 't23', 't24'}

14 Param Declarations
    Elim : Size=1, Index=None, Domain=Any, Default=None, Mutable=True
        Key  : Value
        None :   inf
    Pmax : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :   200
         G2 :   290
         G3 :   190
         G4 :   260
    Pmin : Size=4, Index=gen, Domain=Any, Default=None, Mutable=False
        Key : Value
         G1 :    28
         G2 :    20
         G3 :    30
         G4 :    20
    RD : Size=4, Index=gen, Domain=Any, Default=None, M